# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto (Part 2)

## The previous work of part 1 has been copied in the next block to simplify the reviewing for peers

In [1]:
####Part 1 work:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

#Retrieving Data from Wikipedia Page

# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

#Collecting required information from the soup object

# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

# append the data into the respective lists
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))
        
#Appending lists appropriately to the dataframe

torontodf = pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})

#Performing the preprocessing operations

#Dropping cells with a borough that is "Not assigned"
torontodf.drop(torontodf[ torontodf['Borough'] == "Not assigned"].index , inplace=True)

# group neighborhoods in the same borough
torontodf = torontodf.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))

# for Neighborhood="Not assigned", make the value the same as Borough
for index, row in torontodf.iterrows():
    if (row["Neighborhood"] == "Not assigned"):
        row["Neighborhood"] = row["Borough"]

## Loading Geograhical Coordinates file from Coursera

In [2]:
coordinatesdf = pd.read_csv("http://cocl.us/Geospatial_data")

# rename the column "PostalCode" to match with previous dataframe
coordinatesdf.rename(columns={"Postal Code": "PostalCode"}, inplace=True)

In [3]:
#merging two dataframes
torontowithcordinatesdf=torontodf.merge(coordinatesdf, on="PostalCode", how="left")

In [4]:
#Displaying top 11 rows
torontowithcordinatesdf.head(11)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [5]:
%%capture
###Testing whether it is the same as required by the question(Suppressing the ouput display after successful verification)

column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(torontowithcordinatesdf[torontowithcordinatesdf["PostalCode"]==postcode], ignore_index=True)
    
test_df